In [ ]:
import mxnet as mx

from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd

from mxnet.gluon import nn 

from mxnet.gluon import data as gdata
from mxnet.gluon.data.vision import transforms as gtransforms

import gluoncv as gcv
from gluoncv import model_zoo 
from gluoncv import data as gcvdata
from gluoncv.utils import TrainingHistory

import time

In [ ]:
# 设备个数
gpu_num = 1
ctx = [mx.gpu(i) for i in range(gpu_num)]

batch_size_per_pgu = 8
batch_size = batch_size_per_pgu * gpu_num

num_workers = 4

In [ ]:
# 注意将模型加载到 GPU 而不是 CPU
#yolov3_model = model_zoo.get_model('yolo3_darknet53_coco', pretrained=True, ctx=ctx)

yolov3_model = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True, ctx=ctx)

## 冻结原有层的权重
#resnet18_cifar10[0].collect_params().setattr('grad_req', 'null')
## 初始化自定义层的权重
#resnet18_cifar10[1].initialize(init=mx.init.Xavier(), ctx=ctx)

只有模型需要加载到 GPU ，训练样本和测试样本并不需要加载到 GPU

In [ ]:


#gcvdata.COCODetection(
#    ["root='~/.mxnet/datasets/coco'", "splits=('instances_val2017',)", 'transform=None', 'min_object_area=0', 'skip_empty=True', 'use_crowd=True'],
#)

coco_train = gcvdata.COCODetection(root='~/data/coco', splits=['instances_train2017'], transform=None)

coco_val = gcvdata.COCODetection(root='~/data/coco', splits=['instances_val2017'], transform=None)

# train_dataset = gcvdata.COCOInstance(root='~/data/coco', splits=['instances_val2017'], transform=None)

In [ ]:
voc_train = gcvdata.VOCDetection(root='~/data/VOC/VOCdevkit', 
                                 #splits=[(2007, 'trainval'), (2012, 'trainval')], 
                                 splits=[(2007, 'trainval')],
                                 transform=None)

voc_val = gcvdata.VOCDetection(root='~/data/VOC/VOCdevkit',
                           splits=[(2007, 'test')],
                           transform=None)

In [ ]:
from gluoncv.data.transforms import presets

transforms_train = presets.yolo.YOLO3DefaultTrainTransform(416,416,yolov3_model)
transforms_val = presets.yolo.YOLO3DefaultValTransform(416,416,yolov3_model)



In [ ]:
transform_train = gtransforms.Compose([
    # Randomly crop an area, and then resize it to be 32x32
    gtransforms.RandomResizedCrop(32),
    # Randomly flip the image horizontally
    gtransforms.RandomFlipLeftRight(),
    # Randomly jitter the brightness, contrast and saturation of the image
    gtransforms.RandomColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    # Randomly adding noise to the image
    gtransforms.RandomLighting(0.1),
    # Transpose the image from height*width*num_channels to num_channels*height*width
    # and map values from [0, 255] to [0,1]
    gtransforms.ToTensor(),
    # Normalize the image with mean and standard deviation calculated across all images
    gtransforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

transform_val = gtransforms.Compose([
    gtransforms.Resize(32),
    gtransforms.ToTensor(),
    gtransforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

In [ ]:
# gluoncv 也提供的转换的接口
from gluoncv.data import transforms as gcvtransforms

gcvtransforms.bbox.crop([12,34,56,78])

gcvtransforms.block.RandomCrop(32)

gcvtransforms.mask.flip([12,34],(2,3))

gcvtransforms.image.random_pca_lighting(nd.array([[[1,2,3],[4,5,6]], [[7,8,9],[10,11,12]]]), 0.6)

In [ ]:
# Dataset 类提供了两个转换函数： transform_first 和 transform ；
# transform_first 只变换 data ； transform 同时变换样本和标签（一个样本的所有数据）

loader_train = gdata.DataLoader(voc_train.transform(transforms_train), 
                              batch_size=batch_size, 
                              shuffle=True, 
                              last_batch='discard', 
                              num_workers=num_workers)

loader_val = gdata.DataLoader(voc_val.transform(transforms_val),
                            batch_size=batch_size,
                            shuffle=False, 
                            num_workers=num_workers)

最优化方法，学习速率调节

In [ ]:
optimizer = mx.optimizer.Adam(learning_rate=0.0001,
                             beta1=0.9,
                             beta2=0.999,
                             epsilon=1e-08,
                             lazy_update=True)

trainer = gluon.Trainer(yolov3_model.collect_params(), optimizer)


# 调节学习速率衰减的倍数
lr_decay = 0.1
lr_decay_epochs_set = set([200, 400])

def update_learn_rate(trainer, epoch, lr_decay_epochs_set, lr_decay=0.1):
    if epoch in lr_decay_epochs_set:
        trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        
        
        
train_history = TrainingHistory(['train', 'val'])

In [ ]:
import gluoncv as gcv
yolov3_loss = gcv.loss.YOLOV3Loss()

In [ ]:
# 第一个参数是 pred ，第二个参数是 label
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

metric = mx.metric.Accuracy()

def metric_val(net, dataset_val, metric=None, ctx=mx.cpu(0)):
    
    if metric is None:
        metric = mx.metric.Accuracy()
        
    for _, batch in enumerate(dataset_val):
        # 将数据切片分别加载到不同的设备上
        data_val = gluon.utils.split_and_load(batch[0], ctx, batch_axis=0)
        label_val = gluon.utils.split_and_load(batch[1], ctx, batch_axis=0)
        
        yhat = [net(x) for x in data_val]
        
        # 第一个参数是 label ， 第二个参数是 output
        metric.update(label_val, yhat)    
  
    return metric.get()

In [ ]:
from gluoncv.data.batchify import Tuple, Stack, Pad

# return stacked images, center_targets, scale_targets, gradient weights, objectness_targets, class_targets
# additionally, return padded ground truth bboxes, so there are 7 components returned by dataloader
batchify_fn = Tuple(*([Stack() for _ in range(6)] + [Pad(axis=0, pad_val=-1) for _ in range(1)]))

loader_train = gdata.DataLoader(voc_train.transform(transforms_train), 
                              batch_size=batch_size, 
                              shuffle=True,
                              batchify_fn=batchify_fn,
                              last_batch='discard', 
                              num_workers=num_workers)

for ib, batch in enumerate(loader_train):
    if ib > 3:
        break
    print('data 0:', batch[0][0].shape, 'label 0:', batch[1][0].shape)
    print('data 1:', batch[0][1].shape, 'label 1:', batch[1][1].shape)

In [ ]:
for ib, batch in enumerate(loader_train):
    if ib > 0:
        break
    print('data:', batch[0][0].shape)
    print('label:', batch[6][0].shape)
    # 将数据切片分别加载到不同的设备上 ； 始终卡在这里，下一条打印信息没有出来
    data_train = gluon.utils.split_and_load(batch, ctx_list=ctx, batch_axis=0)
    
    print("aaa")

    with autograd.record():
        input_order = [0, 6, 1, 2, 3, 4, 5]
        obj_loss, center_loss, scale_loss, cls_loss = yolov3_model(*[data_train[o] for o in input_order])
        # sum up the losses
        # some standard gluon training steps:
        # autograd.backward(sum_loss)
        # trainer.step(batch_size)
        print("bbb")

In [ ]:
def train_net(net, data_train, data_val, trainer, epochs):
    
    
    for epoch in range(epochs):
        
        #metric.reset()
        train_loss = 0
        update_learn_rate(trainer, epoch, lr_decay_epochs, lr_decay)
        tic = time.time()
        
        # dataset 是可迭代对象
        for i, batch in enumerate(loader_train):
            # 将数据切片分别加载到不同的设备上
            data_train = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            label_train = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
            
            # 自动求导，是因为这里的处理才导致 list 有 backward 函数？
            with autograd.record():
                output = [net(X) for X in data_train]
                loss = [loss_fn(yhat, y) for yhat, y in zip(output, label_train)]                


            for l in loss:
                # loss 是一个 list ，为什么可以迭代？ 而且这里的 l 和 loss 是一样的？ 
                l.backward()
            
            # 这里更新的时候是怎样用到上面计算的 loss 的？
            # trainer 已经中指定了需要训练的参数
            trainer.step(batch_size)
                        
            for l in loss:
                train_loss += l.sum().asscalar() / batch_size
            
            # 每个 batch 更新一下训练的准确率
            metric.update(label_train, output)
            
        _, acc = metric.get()
        _, val_acc = metric_val(net, dataset_val, ctx=ctx)
        
        # 这里记录的是错误率
        train_history.update([1-acc, 1-val_acc])
        
        toc = time.time()
        
        print('[epoch %d] train_loss=%f, acc=%f, val_acc=%f, lr=%.9f, time: %fs' % 
              (epoch, train_loss, acc, val_acc, trainer.learning_rate, toc-tic))


In [ ]:
train_net(resnet20, dataset_train, dataset_val, trainer)

In [ ]:
%matplotlib inline
train_history.plot()

In [ ]:
resnet20.save_parameters('cifar10_resnet20v2.params')

In [ ]:
resnet20.load_parameters('cifar10_resnet20v2.params', ctx=ctx)

In [ ]:
Signature:      loss(*args)
Type:           YOLOV3Loss
String form:    YOLOV3Loss(batch_axis=0, w=None)
File:           ~/anaconda3/lib/python3.7/site-packages/gluoncv/loss.py
Source:
    
class YOLOV3Loss(gluon.loss.Loss):
    """Losses of YOLO v3.

    Parameters
    ----------
    batch_axis : int, default 0
        The axis that represents mini-batch.
    weight : float or None
        Global scalar weight for loss.

    """
    def __init__(self, batch_axis=0, weight=None, **kwargs):
        super(YOLOV3Loss, self).__init__(weight, batch_axis, **kwargs)
        self._sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
        self._l1_loss = gluon.loss.L1Loss()

    def hybrid_forward(self, F, objness, box_centers, box_scales, cls_preds,
                       objness_t, center_t, scale_t, weight_t, class_t, class_mask):
        """Compute YOLOv3 losses.

        Parameters
        ----------
        objness : mxnet.nd.NDArray
            Predicted objectness (B, N), range (0, 1).
        box_centers : mxnet.nd.NDArray
            Predicted box centers (x, y) (B, N, 2), range (0, 1).
        box_scales : mxnet.nd.NDArray
            Predicted box scales (width, height) (B, N, 2).
        cls_preds : mxnet.nd.NDArray
            Predicted class predictions (B, N, num_class), range (0, 1).
        objness_t : mxnet.nd.NDArray
            Objectness target, (B, N), 0 for negative 1 for positive, -1 for ignore.
        center_t : mxnet.nd.NDArray
            Center (x, y) targets (B, N, 2).
        scale_t : mxnet.nd.NDArray
            Scale (width, height) targets (B, N, 2).
        weight_t : mxnet.nd.NDArray
            Loss Multipliers for center and scale targets (B, N, 2).
        class_t : mxnet.nd.NDArray
            Class targets (B, N, num_class).
            It's relaxed one-hot vector, i.e., (1, 0, 1, 0, 0).
            It can contain more than one positive class.
        class_mask : mxnet.nd.NDArray
            0 or 1 mask array to mask out ignored samples (B, N, num_class).

        Returns
        -------
        tuple of NDArrays
            obj_loss: sum of objectness logistic loss
            center_loss: sum of box center logistic regression loss
            scale_loss: sum of box scale l1 loss
            cls_loss: sum of per class logistic loss

        """
        # compute some normalization count, except batch-size
        denorm = F.cast(
            F.shape_array(objness_t).slice_axis(axis=0, begin=1, end=None).prod(), 'float32')
        weight_t = F.broadcast_mul(weight_t, objness_t)
        hard_objness_t = F.where(objness_t > 0, F.ones_like(objness_t), objness_t)
        new_objness_mask = F.where(objness_t > 0, objness_t, objness_t >= 0)
        obj_loss = F.broadcast_mul(
            self._sigmoid_ce(objness, hard_objness_t, new_objness_mask), denorm)
        center_loss = F.broadcast_mul(self._sigmoid_ce(box_centers, center_t, weight_t), denorm * 2)
        scale_loss = F.broadcast_mul(self._l1_loss(box_scales, scale_t, weight_t), denorm * 2)
        denorm_class = F.cast(
            F.shape_array(class_t).slice_axis(axis=0, begin=1, end=None).prod(), 'float32')
        class_mask = F.broadcast_mul(class_mask, objness_t)
        cls_loss = F.broadcast_mul(self._sigmoid_ce(cls_preds, class_t, class_mask), denorm_class)
        return obj_loss, center_loss, scale_loss, cls_loss
    
    
Call docstring: Calls forward. Only accepts positional arguments.

In [ ]:
from mxnet import gluon

def yolov3loss(objectness_p, box_p, class_p,
              objectness_t, box_t, class_t):
    '''Compute YOLOv3 losses
    
    Parameters
    ---------
    objectness_p : mxnet.nd.NDArray
        Predicted objectness (B, N), range (0, 1).
    box_centers : mxnet.nd.NDArray
        Predicted box centers (x, y) (B, N, 2), range (0, 1).
    box_scales : mxnet.nd.NDArray
        Predicted box scales (width, height) (B, N, 2).
    cls_preds : mxnet.nd.NDArray
        Predicted class predictions (B, N, num_class), range (0, 1).
    objness_t : mxnet.nd.NDArray
        Objectness target, (B, N), 0 for negative 1 for positive, -1 for ignore.
    center_t : mxnet.nd.NDArray
        Center (x, y) targets (B, N, 2).
    scale_t : mxnet.nd.NDArray
        Scale (width, height) targets (B, N, 2).
    weight_t : mxnet.nd.NDArray
        Loss Multipliers for center and scale targets (B, N, 2).
    class_t : mxnet.nd.NDArray
        Class targets (B, N, num_class).
        It's relaxed one-hot vector, i.e., (1, 0, 1, 0, 0).
        It can contain more than one positive class.
    class_mask : mxnet.nd.NDArray
        0 or 1 mask array to mask out ignored samples (B, N, num_class).

    Returns
    -------
    tuple of NDArrays
        obj_loss: sum of objectness logistic loss
        center_loss: sum of box center logistic regression loss
        scale_loss: sum of box scale l1 loss
        cls_loss: sum of per class logistic loss   
    '''
    
    l1_loss = gluon.loss.L1Loss()
    sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntryLoss(from_sigmoid=False)
    
    def SSE(lnum, rnum):
        assert(lnum.shape == rnum.shape)        
        loss = (lnum - rnum) ** 2
    
    box_reg_loss = SSE(box_p, box_t)

In [ ]:
from mxnet import gluon

def yolov3loss(objectness_p, box_p, class_p,
              objectness_t, box_t, class_t):
    '''Compute YOLOv3 losses
    
    Parameters
    ---------
    objectness_p : mxnet.nd.NDArray
        Predicted objectness (B, N), range (0, 1).
    box_centers : mxnet.nd.NDArray
        Predicted box centers (x, y) (B, N, 2), range (0, 1).
    box_scales : mxnet.nd.NDArray
        Predicted box scales (width, height) (B, N, 2).
    cls_preds : mxnet.nd.NDArray
        Predicted class predictions (B, N, num_class), range (0, 1).
    objness_t : mxnet.nd.NDArray
        Objectness target, (B, N), 0 for negative 1 for positive, -1 for ignore.
    center_t : mxnet.nd.NDArray
        Center (x, y) targets (B, N, 2).
    scale_t : mxnet.nd.NDArray
        Scale (width, height) targets (B, N, 2).
    weight_t : mxnet.nd.NDArray
        Loss Multipliers for center and scale targets (B, N, 2).
    class_t : mxnet.nd.NDArray
        Class targets (B, N, num_class).
        It's relaxed one-hot vector, i.e., (1, 0, 1, 0, 0).
        It can contain more than one positive class.
    class_mask : mxnet.nd.NDArray
        0 or 1 mask array to mask out ignored samples (B, N, num_class).

    Returns
    -------
    tuple of NDArrays
        obj_loss: sum of objectness logistic loss
        center_loss: sum of box center logistic regression loss
        scale_loss: sum of box scale l1 loss
        cls_loss: sum of per class logistic loss   
    '''
    
    l1_loss = gluon.loss.L1Loss()
    sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntryLoss(from_sigmoid=False)
    
    def SSE(lnum, rnum):
        assert(lnum.shape == rnum.shape)        
        loss = (lnum - rnum) ** 2
    
    box_reg_loss = SSE(box_p, box_t)
    
    